In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim import Adam

In [12]:
batch_size = 64

In [13]:

#load the data
train_loader = DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [15]:
model = Net()
optimizer = Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

In [17]:
epochs = 1
for epoch in range(epochs):
    losses = 0
    i = 0
    for x, y in train_loader:

        y_hat = model(x)

        loss = loss_fn(y_hat, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses += loss.item()
        i += 1

    print(f'Epoch: {epoch}, loss: {losses / i}')


Epoch: 0, loss: 0.34418514790311294


In [29]:
i = 0
acc = 0

for x_test, y_test in test_loader:

    model.eval()
    with torch.no_grad():
        y_pred = model(x_test)

    y_argmax = y_pred.argmax(1)

    acc += len(torch.nonzero(y_argmax == y_test)) / len(y_test)
    i += 1

print(f'accuracy = {acc / i}')

accuracy = 0.9749203821656051
